# `tensorflow.keras` sweep across pre-trained models


This notebook demonstrates the use of (1) well-known CNN architectures, (2) pretrained weights on `RadImageNet` in a multiclassification task, (3) `wandb.ai` sweeps in comparing the pre-trained architectures.

References:

* The pretrained weights published by [Mei et al (2022)](https://pubs.rsna.org/doi/10.1148/ryai.210315) in their github [link](https://github.com/BMEII-AI/RadImageNet).

* The github repository form the BAGLS team contained in this github [link](https://github.com/anki-xyz/bagls/blob/master/Utils/DataGenerator.py#L109)


In [14]:
# dev convenience
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import sys
sys.path.append("..")
import PATHS

import os
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["WANDB_SILENT"] = "True"
os.environ["WANDB_NOTEBOOK_NAME"] = "05-pretrained-sweep.ipynb"

PROJECT_NAME = 'bagls-sh-test'
GROUP_NAME = 'pretrained-models-sweep'
METRICS_TABLE_NAME = 'metrics_table'

In [16]:
import wandb
print("W&B: ", wandb.__version__)
wandb.login()

# # manage logs
# import logging

# logger = logging.getLogger("wandb")
# logger.setLevel(logging.ERROR)

# logging.getLogger('tensorflow').disabled = True

W&B:  0.13.5


True

In [17]:
# tf loader
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow import keras

In [18]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9055340085874400362
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 6532517587126003743
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1"
]


2022-11-29 09:34:34.277810: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1


In [19]:
import config
configs = config.nb_configs
configs.update(config.pt_configs)

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import preprocess_input

# initialize data generator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=configs["validation_split"],
    rescale=configs["rescale"],
    width_shift_range=configs["width_shift_range"],
    height_shift_range=configs["height_shift_range"],
    shear_range=configs["shear_range"],
    zoom_range=configs["zoom_range"],
    fill_mode=configs["fill_mode"],
    horizontal_flip=configs["horizontal_flip"],
    rotation_range=configs["rotation_range"],
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    rescale=configs["rescale"],
)

In [21]:
train_dir = configs["train_dir"]
test_dir = configs["test_dir"]

batch_size = configs["batch_size"]
class_names = configs["class_names"]
interpol = configs["interpol"]
cmap = configs["cmap"]
label_mode = configs["label_mode"]
labels = configs["labels"]
image_size = configs["image_size"]


train_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="training",
)

val_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="validation",
)

test_dataset = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    shuffle=False, # do not shuffle for later evaluation, alphanum sort
)

configs.update({"val_steps": val_dataset.samples // configs["batch_size"]})

Found 526 images belonging to 2 classes.
Found 26 images belonging to 2 classes.
Found 33 images belonging to 2 classes.


In [22]:
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras import losses

In [23]:
def define_model(model_arch, weights):
    conv_base = model_arch(
        weights=weights,
        include_top=configs["include_top"],
        input_shape=configs["input_shape"],
        pooling=configs["pooling"],
    )
    # freeze conv base weights
    conv_base.trainable = False
    x = conv_base.output
    
    # layers at this stage are arbitrary
    # can be subjected to hyperparam tuning
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=512, activation='relu')(x)
    x = keras.layers.Dropout(configs["dropout_rate"])(x)
    outputs = keras.layers.Dense(units=2, activation="softmax")(x)
    model = keras.Model(inputs=conv_base.input, outputs=outputs)
    return model

In [24]:
from interpretation import ValLog, GRADCamLogger
import utils

def train(model_name):
    GRADCAM_LAYER_NAME = config.pt_gradcam_layers[model_name]
    model_arch = config.pt_models[model_name]
    weights = config.pt_weights[model_name]
    
    tf.keras.backend.clear_session()
    
    thresh = configs["thresh"]
    metrics_dict = {
        "ACC":  metrics.BinaryAccuracy(name="ACC", threshold=thresh),
        "AUC-ROC": metrics.AUC(name='ROC', curve='ROC'),
        "AUC-PR": metrics.AUC(name='PR', curve='PR'),
        "TP": metrics.TruePositives(name="TP", thresholds=thresh),
        "TN": metrics.TrueNegatives(name="TN", thresholds=thresh),
        "FP": metrics.FalsePositives(name="FP", thresholds=thresh),
        "FN": metrics.FalseNegatives(name="FN", thresholds=thresh),
    }

    # opt = optimizers.Adam(learning_rate=1e-06)
    opt = optimizers.Adam()
    met = list(metrics_dict.values())

    model = define_model(model_arch, weights)
    model.compile(
        loss=losses.CategoricalCrossentropy(),
        optimizer=opt,
        metrics=met,
    )
    # initialize run
    run = wandb.init(
        project=PROJECT_NAME, 
        group=GROUP_NAME,
        name=model_name,
        job_type='train',
        config=configs, 
    )

    wandb_callback = wandb.keras.WandbCallback(
        monitor="val_ROC",
        mode="max",
        save_model=True,
        save_graph=True,
        compute_flops=True,
    )

    callbacks = [
        wandb_callback,
        ValLog(generator=val_dataset, num_log_batches=1),
        GRADCamLogger(generator=test_dataset, 
                      layer_name=GRADCAM_LAYER_NAME, num_log_batches=1)
    ]
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=configs["epochs"], 
        shuffle=True,
        callbacks=callbacks,
    )
    run.finish()
    
def evaluate(model_name):
    run = wandb.init(
        project=PROJECT_NAME, 
        group=GROUP_NAME,
        name=model_name,
        job_type="inference-evaluation", 
        config=configs, 
    )
    model_at = run.use_artifact("model-" + model_name + ":latest")
    model_dir = model_at.download()
    best_model = keras.models.load_model(model_dir)
    
    metrics_results = best_model.evaluate(test_dataset)
    metrics_results = dict(zip(["loss"] + list(metrics_dict.keys()), 
                               metrics_results))
    tp, fp, tn, fn = (metrics_results["TP"], metrics_results["FP"], 
                      metrics_results["TN"], metrics_results["FN"])

    add_metrics = {
        "SENSITIVITY": utils.get_sensitivity(tp, fp, tn, fn),
        "SPECIFICTY": utils.get_specificity(tp, fp, tn, fn),
        "PPV": utils.get_ppv(tp, fp, tn, fn),
        "NPV": utils.get_npv(tp, fp, tn, fn),
        "F1" : utils.get_fbeta(tp, fp, tn, fn, beta=1),
    }
    metrics_results.update(add_metrics)

    print(f"Metrics: \n", metrics_results)

    columns = list(metrics_results.keys())
    metrics_table = wandb.Table(columns=columns)
    metrics_table.add_data(*metrics_results.values())
    wandb.run.log({METRICS_TABLE_NAME : metrics_table})

    # add logging of confusion matrix image from matplotlib

    # get preds
    trained_preds = best_model.predict(test_dataset)
    run.finish()

In [25]:
for model_name in config.pt_models.keys():
    print(model_name)
    train(model_name)
    evaluate(model_name)

ResNet50


2022-11-29 09:35:01.375037: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-11-29 09:35:01.375299: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-11-29 09:35:01.378214: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1
2022-11-29 09:35:01.393652: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.008ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



Epoch 1/5
9/9 [==============================] - 16s 1s/step - loss: 0.6522 - ACC: 0.6388 - ROC: 0.6839 - PR: 0.6556 - TP: 336.0000 - TN: 336.0000 - FP: 190.0000 - FN: 190.0000 - val_loss: 0.5301 - val_ACC: 0.8077 - val_ROC: 0.8447 - val_PR: 0.7956 - val_TP: 21.0000 - val_TN: 21.0000 - val_FP: 5.0000 - val_FN: 5.0000


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221129_093439-1njpxc4s/files/model-best/assets


...

Epoch 1/5
9/9 [==============================] - 25s 2s/step - loss: 0.6360 - ACC: 0.6540 - ROC: 0.6946 - PR: 0.6683 - TP: 344.0000 - TN: 344.0000 - FP: 182.0000 - FN: 182.0000 - val_loss: 0.5407 - val_ACC: 0.8077 - val_ROC: 0.7663 - val_PR: 0.7483 - val_TP: 21.0000 - val_TN: 21.0000 - val_FP: 5.0000 - val_FN: 5.0000
INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221129_094358-y5nb8c9u/files/model-best/assets
Epoch 2/5
9/9 [==============================] - 9s 1s/step - loss: 0.6103 - ACC: 0.6692 - ROC: 0.7352 - PR: 0.7092 - TP: 352.0000 - TN: 352.0000 - FP: 174.0000 - FN: 174.0000 - val_loss: 0.5825 - val_ACC: 0.7308 - val_ROC: 0.7692 - val_PR: 0.7621 - val_TP: 19.0000 - val_TN: 19.0000 - val_FP: 7.0000 - val_FN: 7.0000
INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221129_094358-y5nb8c9u/files/model-best/assets
Epoch 3/5
9/9 [==============================] - 9s 1s/step - loss: 0.5771 - ACC:

1/1 [==============================] - 5s 5s/step - loss: 0.3724 - ACC: 0.8788 - ROC: 0.9679 - PR: 0.9700 - TP: 29.0000 - TN: 29.0000 - FP: 4.0000 - FN: 4.0000
Metrics: 
 {'loss': 0.372403085231781, 'ACC': 0.8787878751754761, 'AUC-ROC': 0.9678604602813721, 'AUC-PR': 0.9700316190719604, 'TP': 29.0, 'TN': 29.0, 'FP': 4.0, 'FN': 4.0, 'SENSITIVITY': 0.8787878787878788, 'SPECIFICTY': 0.8787878787878788, 'PPV': 0.8787878787878788, 'NPV': 0.8787878787878788, 'F1': 0.8787878787878788}
InceptionResNetV2


2022-11-29 09:52:23.562184: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-11-29 09:52:23.562507: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-11-29 09:52:23.565765: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1
2022-11-29 09:52:23.647589: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.02ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



Epoch 1/5
9/9 [==============================] - 30s 2s/step - loss: 0.6448 - ACC: 0.6483 - ROC: 0.6936 - PR: 0.6553 - TP: 341.0000 - TN: 341.0000 - FP: 185.0000 - FN: 185.0000 - val_loss: 0.5419 - val_ACC: 0.7308 - val_ROC: 0.8003 - val_PR: 0.8059 - val_TP: 19.0000 - val_TN: 19.0000 - val_FP: 7.0000 - val_FN: 7.0000
INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221129_095211-3s6fgb9a/files/model-best/assets
Epoch 2/5
9/9 [==============================] - 10s 1s/step - loss: 0.6070 - ACC: 0.6977 - ROC: 0.7385 - PR: 0.7066 - TP: 367.0000 - TN: 367.0000 - FP: 159.0000 - FN: 159.0000 - val_loss: 0.5735 - val_ACC: 0.7692 - val_ROC: 0.7589 - val_PR: 0.7226 - val_TP: 20.0000 - val_TN: 20.0000 - val_FP: 6.0000 - val_FN: 6.0000
Epoch 3/5
9/9 [==============================] - 9s 1s/step - loss: 0.5963 - ACC: 0.7072 - ROC: 0.7510 - PR: 0.7255 - TP: 372.0000 - TN: 372.0000 - FP: 154.0000 - FN: 154.0000 - val_loss: 0.5733 - val_ACC: 0.6923 - val_RO

1/1 [==============================] - 6s 6s/step - loss: 0.4262 - ACC: 0.8485 - ROC: 0.9403 - PR: 0.9452 - TP: 28.0000 - TN: 28.0000 - FP: 5.0000 - FN: 5.0000
Metrics: 
 {'loss': 0.42621949315071106, 'ACC': 0.8484848737716675, 'AUC-ROC': 0.9403122663497925, 'AUC-PR': 0.9452385306358337, 'TP': 28.0, 'TN': 28.0, 'FP': 5.0, 'FN': 5.0, 'SENSITIVITY': 0.8484848484848485, 'SPECIFICTY': 0.8484848484848485, 'PPV': 0.8484848484848485, 'NPV': 0.8484848484848485, 'F1': 0.8484848484848485}


## End